# Relaxed Search(제대로) 가미한 코드

아는 선에서 일단 건드려봤고 1차적인 결과를 올림<br>
알고리즘 돌리는 코드들은 동환이형이 쓴 그대로 썼고,<br>
알고리즘에 따른 셀 분리, relaxed search, best model 등을 구하기만 함. <br>
한번 쭉 봐주고 갖고 놀아보고 이것저것 만져보면 뭐가 뭔지 대충 느낌 올거임.<br>
아님 말구<br>
<strong>plot 은 아직 하나도 안 넣음(ex. 동환이형의 시간축 x 그래프). data visualization 에 필수적인 것이니 꼭 넣어야함</strong>
<br><br>

### 5월 23일 업데이트

<strong> 이 코드 예상 실행 시간... 적으면 2분... 많으면 5분?(데스크탑 기준...)</strong><br>
데이터 양에 따라 차이가 남.


In [1]:
# 최적의 패러미터 가진 모델을 몇번 돌릴지 값.
test_times=10

In [2]:
#데이터 파일 업로드
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import scipy.stats
import random
import warnings
np.set_printoptions(precision=4,suppress=True)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)
plt.rcParams["font.family"]='Malgun Gothic'

아래 셀에서 데이터 수정. <strong> 앞에 Data/ 있어야 제대로 됨</strong><br>
원하는 데이터 이름만 집어넣으면 뒤에는 알아서 될거임

In [3]:
dataname = "ilsan_3years_small.csv"

In [4]:
#데이터 확인 및 필요 없는 열 삭제
pd.set_option('display.max_columns', 500)
df=pd.read_csv(dataname)
#print(df.head())

df.drop(['시군구'],axis=1,inplace=True)
df.drop(['단지명'],axis=1,inplace=True)
df.drop(['건축년도'],axis=1,inplace=True)
# df.drop(['마트최소거리'],axis=1,inplace=True)
# df.drop(['지하철도보거리'],axis=1,inplace=True)
# df.drop(['용적률'],axis=1,inplace=True)
# df.drop(['대형병원최소거리'],axis=1,inplace=True)
#df.drop(['층'],axis=1,inplace=True)
# df.drop(['건폐율'],axis=1,inplace=True)
#df.drop(['총세대수'],axis=1,inplace=True)
# df.drop(['규모별월세가격지수'],axis=1,inplace=True)
# df.drop(['규모별수익률'],axis=1,inplace=True)
# df.drop(['동별인구수'],axis=1,inplace=True)
#print(df.head())

#데이터 형태 실수로 변환
#면적 거래금액 분리버전
Y = df['거래금액'].values
Y = Y.astype('float')
X = df.drop(labels=['거래금액'],axis=1)
X = X.astype('float')

# 단위면적당 거래금액 버전
# df.drop(['전용면적'],axis=1,inplace=True)
# df.drop(['거래금액'],axis=1,inplace=True)
# Y = df['거래금액2'].values
# Y = Y.astype('float')
# X = df.drop(labels=['거래금액2'],axis=1)
# X = X.astype('float')

In [5]:
test_size = [0.1,0.15,0.2,0.25,0.3]
ts_len = len(test_size)

### 1. Random Forest

n_estimators = number of trees in the foreset<br>
max_features = max number of features considered for splitting a node<br>
max_depth = max number of levels in each decision tree<br>
min_samples_split = min number of data points placed in a node before the node is split<br>
min_samples_leaf = min number of data points allowed in a leaf node<br>
bootstrap = method for sampling data points (with or without replacement)

In [6]:
Tree_nums = [5,10,50,100,250,500,750]
Var_nums = [1,3,5,7]

# Random Forest Relaxed Search
x = len(Tree_nums)
y = len(Var_nums)
count=0

# best_RF_RMSE_diff=100
# best_RF_RMSE_train=0
# best_RF_RMSE_test=0
# best_RF_MAPE_train=0
# best_RF_MAPE_test=0
RF_RMSE_train_list =[]
RF_RMSE_test_list =[]
RF_MAPE_train_list =[]
RF_MAPE_test_list =[]
RF_par_diff_list=[]
RF_par_tsize_list=[]
RF_par_trees_list=[]
RF_par_var_list=[]
RF_min_vars_tsize_list=[]
RF_min_vars_trees_list=[]
RF_min_vars_var_list=[]   

print("----progress...----")

for ts in test_size:
    for a in Tree_nums:
        for b in Var_nums:
            random.seed(a*10)
            #훈련셋, 시험셋 분류 
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)
        
            #랜덤포레스트
            RF_model = RandomForestRegressor(n_estimators=a, max_depth=b)
            RF_model.fit(X_train, Y_train)
            
            #예측값 생성
            RF_prediction_test = RF_model.predict(X_test)
            RF_prediction_test = RF_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
            RF_train_test = RF_model.predict(X_train)
            RF_train_test = RF_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
            RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
            RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
            RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
            RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)
            
            # 값들을 위에 지정한 list 에다 다 저장
            RF_RMSE_train_list.append(RF_RMSE_train)
            RF_RMSE_test_list.append(RF_RMSE_test)
            RF_MAPE_train_list.append(RF_MAPE_train)
            RF_MAPE_test_list.append(RF_MAPE_test)
            
            RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)
            
            RF_par_diff_list.append(RF_RMSE_diff)
            RF_par_tsize_list.append(ts)
            RF_par_trees_list.append(a)
            RF_par_var_list.append(b)
            
#             if best_RF_RMSE_diff>RF_RMSE_diff:
#                 best_RF_RMSE_diff = RF_RMSE_diff
#                 best_RF_RMSE_train = RF_RMSE_train
#                 best_RF_RMSE_test = RF_RMSE_test
#                 best_RF_MAPE_train = RF_MAPE_train
#                 best_RF_MAPE_test = RF_MAPE_test
#                 RF_C = count+1
#                 RF_TS = ts
#                 RF_A = a
#                 RF_B = b
         
                
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                  .format(progress,count,ts,a,b,RF_RMSE_diff,RF_RMSE_train,RF_RMSE_test,RF_MAPE_train,RF_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  Tree_num: {} , Var_num: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(RF_C,RF_TS,RF_A,RF_B,best_RF_RMSE_train,best_RF_RMSE_test,best_RF_RMSE_diff,best_RF_MAPE_train,best_RF_MAPE_test))
                


for i in range(5):
    val = min(RF_par_diff_list)
    index = RF_par_diff_list.index(val)
    RF_min_vars_tsize_list.append(RF_par_tsize_list.pop(index))
    RF_min_vars_trees_list.append(RF_par_trees_list.pop(index))
    RF_min_vars_var_list.append(RF_par_var_list.pop(index))


----progress...----
 0.71% , case:1 , test_size: 0.1 , Tree_num: 5 , Var_num: 1 , difference(%) : 11.84
 ---  train RMSE : 2739 , test RMSE : 3107
---  train MAPE : 17.58 , test MAPE : 16.76
 1.43% , case:2 , test_size: 0.1 , Tree_num: 5 , Var_num: 3 , difference(%) : 1.63
 ---  train RMSE : 1810 , test RMSE : 1840
---  train MAPE : 11.39 , test MAPE : 11.82
 2.14% , case:3 , test_size: 0.1 , Tree_num: 5 , Var_num: 5 , difference(%) : 8.63
 ---  train RMSE : 1355 , test RMSE : 1483
---  train MAPE : 8.17 , test MAPE : 8.83
 2.86% , case:4 , test_size: 0.1 , Tree_num: 5 , Var_num: 7 , difference(%) : 22.16
 ---  train RMSE : 980 , test RMSE : 1259
---  train MAPE : 5.73 , test MAPE : 7.9
 3.57% , case:5 , test_size: 0.1 , Tree_num: 10 , Var_num: 1 , difference(%) : 1.15
 ---  train RMSE : 2751 , test RMSE : 2783
---  train MAPE : 17.41 , test MAPE : 17.62
 4.29% , case:6 , test_size: 0.1 , Tree_num: 10 , Var_num: 3 , difference(%) : 12.36
 ---  train RMSE : 1863 , test RMSE : 1658
---  

In [11]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("test_size : {} , Tree_num : {}, Var_num : {}".format(RF_min_vars_tsize_list[i],RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))

print("...testing...")

progress=0
count=0
RF_Best_RMSE_train_list=[]
RF_Best_RMSE_test_list=[]
RF_Best_MAPE_train_list=[]
RF_Best_MAPE_test_list=[]
RF_Best_RMSE_diff_list=[]

RF_Best_total_data=[]

for i in range(5):
    temp_RF_Best_RMSE_train_list=[]
    temp_RF_Best_RMSE_test_list=[]
    temp_RF_Best_MAPE_train_list=[]
    temp_RF_Best_MAPE_test_list=[]
    temp_RF_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = RF_min_vars_tsize_list[i]
        a = RF_min_vars_trees_list[i]
        b = RF_min_vars_var_list[i]
        random.seed((k+1)*10)

        #훈련셋, 시험셋 분류 
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        #랜덤포레스트
        RF_Best_model = RandomForestRegressor(n_estimators=a, max_depth=b, max_features=9)
        RF_Best_model.fit(X_train, Y_train)

            #예측값 생성
        RF_Best_prediction_test = RF_Best_model.predict(X_test)
        RF_Best_prediction_test = RF_Best_prediction_test.astype('int')

            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
        RF_Best_train_test = RF_Best_model.predict(X_train)
        RF_Best_train_test = RF_Best_train_test.astype('int')

            #훈련셋 RMSE & MAPE
        RF_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_Best_train_test)).astype('int')
        RF_Best_MAPE_train = round(np.mean(np.abs((Y_train - RF_Best_train_test) / Y_train)) * 100 ,2)

            #시험셋 RMSE & MAPE
        RF_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_Best_prediction_test)).astype('int')
        RF_Best_MAPE_test = round(np.mean(np.abs((Y_test - RF_Best_prediction_test) / Y_test)) * 100 ,2)

        RF_Best_RMSE_diff = np.abs((RF_Best_RMSE_test-RF_Best_RMSE_train)/RF_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_RF_Best_RMSE_train_list.append(RF_Best_RMSE_train)
        temp_RF_Best_RMSE_test_list.append(RF_Best_RMSE_test)
        temp_RF_Best_MAPE_train_list.append(RF_Best_MAPE_train)
        temp_RF_Best_MAPE_test_list.append(RF_Best_MAPE_test)
        temp_RF_Best_RMSE_diff_list.append(RF_Best_RMSE_diff)
        
        
        count+=1
        progress = round(count/(10*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,a,b,RF_Best_RMSE_diff,RF_Best_RMSE_train,RF_Best_RMSE_test,RF_Best_MAPE_train,RF_Best_MAPE_test))
    
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_diff_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_diff_list))
    
#     RF_Best_RMSE_train_list.append(temp_RF_Best_RMSE_train_list)
#     RF_Best_RMSE_test_list.append(temp_RF_Best_RMSE_test_list)
#     RF_Best_MAPE_train_list.append(temp_RF_Best_MAPE_train_list)
#     RF_Best_MAPE_test_list.append(temp_RF_Best_MAPE_test_list)
#     RF_Best_RMSE_diff_list.append(temp_RF_Best_RMSE_diff_list)
    
    
# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} , Tree_num: {} , Var_num: {} ,   ---  MAPE : {}".format(RF_Best_C,RF_Best_TS,RF_Best_A,RF_Best_B,RF_Best_min))

PARAMETERS FOR BEST RF...
test_size : 0.15 , Tree_num : 50, Var_num : 7
test_size : 0.15 , Tree_num : 100, Var_num : 1
test_size : 0.15 , Tree_num : 100, Var_num : 3
test_size : 0.15 , Tree_num : 100, Var_num : 5
test_size : 0.15 , Tree_num : 100, Var_num : 7
...testing...
 2.00% , case:1 , test_size: 0.15 , Tree_num: 50 , Var_num: 7 , difference(%) : 19.42
 ---  train RMSE : 946 , test RMSE : 1174
---  train MAPE : 5.55 , test MAPE : 6.72
 4.00% , case:2 , test_size: 0.15 , Tree_num: 50 , Var_num: 7 , difference(%) : 21.11
 ---  train RMSE : 942 , test RMSE : 1194
---  train MAPE : 5.76 , test MAPE : 6.43
 6.00% , case:3 , test_size: 0.15 , Tree_num: 50 , Var_num: 7 , difference(%) : 58.44
 ---  train RMSE : 884 , test RMSE : 2127
---  train MAPE : 5.45 , test MAPE : 8.06
 8.00% , case:4 , test_size: 0.15 , Tree_num: 50 , Var_num: 7 , difference(%) : 21.87
 ---  train RMSE : 943 , test RMSE : 1207
---  train MAPE : 5.52 , test MAPE : 6.45
 10.00% , case:5 , test_size: 0.15 , Tree_num:

In [12]:
rf1=[]
rf2=[]
rf3=[]
rf4=[]
rf5=[]

for i in range(50):
    k=i//10
    if k==0:
        rf1.append(RF_Best_total_data[i])
    if k==1:
        rf2.append(RF_Best_total_data[i])
    if k==2:
        rf3.append(RF_Best_total_data[i])
    if k==3:
        rf4.append(RF_Best_total_data[i])
    if k==4:
        rf5.append(RF_Best_total_data[i])
    
RF_s1 = pd.Series(rf1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

RF_s2 = pd.Series(rf2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
RF_s3 = pd.Series(rf3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
RF_s4 = pd.Series(rf4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
RF_s5 = pd.Series(rf5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

RF_best_total_dataframe = pd.concat([RF_s1,RF_s2,RF_s3,RF_s4,RF_s5],axis=1)
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,940.700000,2676.800000,1779.100000,1313.900000,928.500000
RMSE test Mean,1331.000000,2562.500000,1859.900000,1427.800000,1332.900000
RMSE train Stdev,32.854376,25.887449,16.287725,13.874077,20.616741
RMSE test Stdev,389.263921,133.844126,167.202542,143.441835,295.482131
MAPE train Mean,5.606000,16.769000,11.412000,8.056000,5.594000
MAPE test Mean,6.816000,16.688000,11.589000,8.567000,6.568000
MAPE train Stdev,0.130706,0.172189,0.105338,0.070456,0.088340
MAPE test Stdev,0.577394,0.861926,0.555670,0.595131,0.274875
RMSE diff(%) Mean,24.201941,6.398911,5.782680,9.795934,27.138025
RMSE diff(%) Stdev,17.752031,4.402686,6.951099,7.027094,14.807495


### 2. Support Vector Machine parameters

In [9]:
gamma_nums = [0.00015,0.001,0.01,0.1]
C_values= [500,1000,5000,10000,50000]
x = len(gamma_nums)
y = len(C_values)
count=0
# Random Forest Relaxed Search
count=0

# best_SVR_RMSE_diff=100
# best_SVR_RMSE_train=0
# best_SVR_RMSE_test=0
# best_SVR_MAPE_train=0
# best_SVR_MAPE_test=0
SVR_RMSE_train_list =[]
SVR_RMSE_test_list =[]
SVR_MAPE_train_list =[]
SVR_MAPE_test_list =[]
SVR_par_diff_list=[]
SVR_par_tsize_list=[]
SVR_par_gamma_list=[]
SVR_par_cval_list=[]
SVR_min_vars_tsize_list=[]
SVR_min_vars_gamma_list=[]
SVR_min_vars_cval_list=[]   

print("----progress...----")

for ts in test_size:
    for a in gamma_nums:
        for b in C_values:
            random.seed(a*10)
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

            SVR_model = svm.SVR(kernel='rbf',gamma=a,C=b)
            SVR_model.fit(X_train, Y_train)
            
            #예측값 생성
            SVR_prediction_test = SVR_model.predict(X_test)
            SVR_prediction_test = SVR_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
            SVR_train_test = SVR_model.predict(X_train)
            SVR_train_test = SVR_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
            SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
            SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
            SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
            SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
            # 값들을 위에 지정한 list 에다 다 저장
            SVR_RMSE_train_list.append(SVR_RMSE_train)
            SVR_RMSE_test_list.append(SVR_RMSE_test)
            SVR_MAPE_train_list.append(SVR_MAPE_train)
            SVR_MAPE_test_list.append(SVR_MAPE_test)
            
            SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
            SVR_par_diff_list.append(SVR_RMSE_diff)
            SVR_par_tsize_list.append(ts)
            SVR_par_gamma_list.append(a)
            SVR_par_cval_list.append(b)
            
#             if best_SVR_RMSE_diff>SVR_RMSE_diff:
#                 best_SVR_RMSE_diff = SVR_RMSE_diff
#                 best_SVR_RMSE_train = SVR_RMSE_train
#                 best_SVR_RMSE_test = SVR_RMSE_test
#                 best_SVR_MAPE_train = SVR_MAPE_train
#                 best_SVR_MAPE_test = SVR_MAPE_test
#                 SVR_C = count+1
#                 SVR_TS = ts
#                 SVR_A = a
#                 SVR_B = b
         
                
            count+=1
            progress = round(count/(x*y*ts_len)*100,2)
            
            print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
            "---  train RMSE : {} , test RMSE : {}\n"
            "---  train MAPE : {} , test MAPE : {}"
                  .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  Gamma_size: {} ,  C_values: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(SVR_C,SVR_TS,SVR_A,SVR_B,best_SVR_RMSE_train,best_SVR_RMSE_test,best_SVR_RMSE_diff,best_SVR_MAPE_train,best_SVR_MAPE_test))
                


for i in range(5):
    val = min(SVR_par_diff_list)
    index = SVR_par_diff_list.index(val)
    SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
    SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
    SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


----progress...----
 1.00% , case:1 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 500 , difference(%) : 3.64
 ---  train RMSE : 2105 , test RMSE : 2031
---  train MAPE : 9.53 , test MAPE : 9.22
 2.00% , case:2 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 1000 , difference(%) : 13.33
 ---  train RMSE : 1820 , test RMSE : 1606
---  train MAPE : 7.71 , test MAPE : 9.58
 3.00% , case:3 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 5000 , difference(%) : 11.97
 ---  train RMSE : 1413 , test RMSE : 1262
---  train MAPE : 5.8 , test MAPE : 7.02
 4.00% , case:4 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 10000 , difference(%) : 2.27
 ---  train RMSE : 1204 , test RMSE : 1232
---  train MAPE : 5.21 , test MAPE : 6.28
 5.00% , case:5 , test_size: 0.1 , Gamma_size: 0.00015 ,  C_values: 50000 , difference(%) : 9.49
 ---  train RMSE : 916 , test RMSE : 1012
---  train MAPE : 4.39 , test MAPE : 6.04
 6.00% , case:6 , test_size: 0.1 , Gamma_size: 0.001 ,  C_values: 500 , di

In [10]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

print("\n\n...testing...\n")
progress=0
count=0
SVR_Best_RMSE_train_list=[]
SVR_Best_RMSE_test_list=[]
SVR_Best_MAPE_train_list=[]
SVR_Best_MAPE_test_list=[]
SVR_Best_RMSE_diff_list=[]

SVR_Best_total_data=[]

for i in range(5):
    temp_SVR_Best_RMSE_train_list=[]
    temp_SVR_Best_RMSE_test_list=[]
    temp_SVR_Best_MAPE_train_list=[]
    temp_SVR_Best_MAPE_test_list=[]
    temp_SVR_Best_RMSE_diff_list=[]
    for k in range(test_times):
        ts = SVR_min_vars_tsize_list[i]
        a = SVR_min_vars_gamma_list[i]
        b = SVR_min_vars_cval_list[i]
        random.seed((k+1)*10)

        #훈련셋, 시험셋 분류 
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        SVR_Best_model = svm.SVR(kernel='rbf',gamma=a,C=b)
        SVR_Best_model.fit(X_train, Y_train)
            
            #예측값 생성
        SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
        SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
            #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
        SVR_Best_train_test = SVR_Best_model.predict(X_train)
        SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
            #훈련셋 RMSE & MAPE
        SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
        SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
            #시험셋 RMSE & MAPE
        SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
        SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
        SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
        temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
        temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
        temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
        temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

    #     if SVR_Best_min>SVR_Best_MAPE_test:
    #         SVR_Best_min = SVR_Best_MAPE_test
    #         SVR_Best_TS = ts
    #         SVR_Best_A = a
    #         SVR_Best_B = b
    #         SVR_Best_C = count+1

        count+=1
        progress = round(count/(test_times*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))

    
    


PARAMETERS FOR BEST SVR...
test_size : 0.2 , Gamma_size: 0.01 ,  C_values: 500
test_size : 0.2 , Gamma_size: 0.01 ,  C_values: 1000
test_size : 0.2 , Gamma_size: 0.01 ,  C_values: 5000
test_size : 0.2 , Gamma_size: 0.01 ,  C_values: 10000
test_size : 0.2 , Gamma_size: 0.01 ,  C_values: 50000


...testing...

 2.00% , case:1 , test_size: 0.2 , Tree_num: 0.01 , Var_num: 500 , difference(%) : 2.27
 ---  train RMSE : 2206 , test RMSE : 2157
---  train MAPE : 8.31 , test MAPE : 10.68
 4.00% , case:2 , test_size: 0.2 , Tree_num: 0.01 , Var_num: 500 , difference(%) : 38.16
 ---  train RMSE : 1907 , test RMSE : 3084
---  train MAPE : 8.28 , test MAPE : 10.08
 6.00% , case:3 , test_size: 0.2 , Tree_num: 0.01 , Var_num: 500 , difference(%) : 13.70
 ---  train RMSE : 2224 , test RMSE : 1956
---  train MAPE : 8.31 , test MAPE : 10.46
 8.00% , case:4 , test_size: 0.2 , Tree_num: 0.01 , Var_num: 500 , difference(%) : 2.65
 ---  train RMSE : 2207 , test RMSE : 2150
---  train MAPE : 8.25 , test MAPE 

In [13]:
SVR1=[]
SVR2=[]
SVR3=[]
SVR4=[]
SVR5=[]

for i in range(50):
    k=i//10
    if k==0:
        SVR1.append(SVR_Best_total_data[i])
    if k==1:
        SVR2.append(SVR_Best_total_data[i])
    if k==2:
        SVR3.append(SVR_Best_total_data[i])
    if k==3:
        SVR4.append(SVR_Best_total_data[i])
    if k==4:
        SVR5.append(SVR_Best_total_data[i])

    
SVR_s1 = pd.Series(SVR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

SVR_s2 = pd.Series(SVR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
SVR_s3 = pd.Series(SVR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
SVR_s4 = pd.Series(SVR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
SVR_s5 = pd.Series(SVR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

SVR_best_total_dataframe_rbf = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
print("SVR best 5 models")
SVR_best_total_dataframe_rbf

SVR best 5 models


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,2142.100000,1820.700000,1038.300000,758.100000,408.400000
RMSE test Mean,2329.600000,2114.800000,1663.000000,1716.600000,1612.100000
RMSE train Stdev,99.158913,97.497744,116.549603,123.733948,13.668943
RMSE test Stdev,324.331682,291.484751,310.446131,307.205208,369.033725
MAPE train Mean,8.337000,6.042000,2.472000,1.830000,1.089000
MAPE test Mean,10.206000,8.823000,6.619000,6.803000,7.119000
MAPE train Stdev,0.071141,0.083042,0.076000,0.064343,0.045924
MAPE test Stdev,0.330369,0.524024,0.464703,0.474005,0.600241
RMSE diff(%) Mean,11.325153,13.134318,34.438686,53.394666,73.446293
RMSE diff(%) Stdev,11.696592,13.273913,16.839834,13.765376,5.621002


### EXTRA... SVM LINEAR KERNEL

In [12]:
# gamma_nums = [0.001,0.01,0.1,1]
# C_values= [10,100,1000,10000]
# x = len(gamma_nums)
# y = len(C_values)
# count=0
# # Random Forest Relaxed Search
# count=0

# # best_SVR_RMSE_diff=100
# # best_SVR_RMSE_train=0
# # best_SVR_RMSE_test=0
# # best_SVR_MAPE_train=0
# # best_SVR_MAPE_test=0
# SVR_RMSE_train_list =[]
# SVR_RMSE_test_list =[]
# SVR_MAPE_train_list =[]
# SVR_MAPE_test_list =[]
# SVR_par_diff_list=[]
# SVR_par_tsize_list=[]
# SVR_par_gamma_list=[]
# SVR_par_cval_list=[]
# SVR_min_vars_tsize_list=[]
# SVR_min_vars_gamma_list=[]
# SVR_min_vars_cval_list=[]   

# print("----progress...----")

# for ts in test_size:
#     for a in gamma_nums:
#         for b in C_values:
#             random.seed(a*10)
#             X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#             SVR_model = svm.SVR(kernel='linear',gamma=a,C=b)
#             SVR_model.fit(X_train, Y_train)
            
#             #예측값 생성
#             SVR_prediction_test = SVR_model.predict(X_test)
#             SVR_prediction_test = SVR_prediction_test.astype('int')
            
#             #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
#             SVR_train_test = SVR_model.predict(X_train)
#             SVR_train_test = SVR_train_test.astype('int')
            
#             #훈련셋 RMSE & MAPE
#             SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
#             SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            
#             #시험셋 RMSE & MAPE
#             SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
#             SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            
#             # 값들을 위에 지정한 list 에다 다 저장
#             SVR_RMSE_train_list.append(SVR_RMSE_train)
#             SVR_RMSE_test_list.append(SVR_RMSE_test)
#             SVR_MAPE_train_list.append(SVR_MAPE_train)
#             SVR_MAPE_test_list.append(SVR_MAPE_test)
            
#             SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
#             SVR_par_diff_list.append(SVR_RMSE_diff)
#             SVR_par_tsize_list.append(ts)
#             SVR_par_gamma_list.append(a)
#             SVR_par_cval_list.append(b)
            
# #             if best_SVR_RMSE_diff>SVR_RMSE_diff:
# #                 best_SVR_RMSE_diff = SVR_RMSE_diff
# #                 best_SVR_RMSE_train = SVR_RMSE_train
# #                 best_SVR_RMSE_test = SVR_RMSE_test
# #                 best_SVR_MAPE_train = SVR_MAPE_train
# #                 best_SVR_MAPE_test = SVR_MAPE_test
# #                 SVR_C = count+1
# #                 SVR_TS = ts
# #                 SVR_A = a
# #                 SVR_B = b
         
                
#             count+=1
#             progress = round(count/(x*y*ts_len)*100,2)
            
#             print(" {:.2f}% , case:{} , test_size: {} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))
            

# # print("\n\n")
# # print("Best Case")
# # print("case:{} , test_size: {} ,  Gamma_size: {} ,  C_values: {} ,  \n"
# #       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
# #       "---  train MAPE : {} , test MAPE : {}"
# #       .format(SVR_C,SVR_TS,SVR_A,SVR_B,best_SVR_RMSE_train,best_SVR_RMSE_test,best_SVR_RMSE_diff,best_SVR_MAPE_train,best_SVR_MAPE_test))
                


# for i in range(5):
#     val = min(SVR_par_diff_list)
#     index = SVR_par_diff_list.index(val)
#     SVR_min_vars_tsize_list.append(SVR_par_tsize_list.pop(index))
#     SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
#     SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))


In [13]:
# print("PARAMETERS FOR BEST SVR...")
# for i in range(5):
#     print("test_size : {} , Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_tsize_list[i],SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

# print("\n\n...testing...\n")
# test_times=5
# progress=0
# count=0
# SVR_Best_RMSE_train_list=[]
# SVR_Best_RMSE_test_list=[]
# SVR_Best_MAPE_train_list=[]
# SVR_Best_MAPE_test_list=[]
# SVR_Best_RMSE_diff_list=[]

# SVR_Best_total_data=[]

# for i in range(5):
#     temp_SVR_Best_RMSE_train_list=[]
#     temp_SVR_Best_RMSE_test_list=[]
#     temp_SVR_Best_MAPE_train_list=[]
#     temp_SVR_Best_MAPE_test_list=[]
#     temp_SVR_Best_RMSE_diff_list=[]
#     for k in range(test_times):
#         ts = SVR_min_vars_tsize_list[i]
#         a = SVR_min_vars_gamma_list[i]
#         b = SVR_min_vars_cval_list[i]
#         random.seed((k+1)*10)

#         #훈련셋, 시험셋 분류 
#         X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

#         SVR_Best_model = svm.SVR(kernel='linear',gamma=a,C=b)
#         SVR_Best_model.fit(X_train, Y_train)
            
#             #예측값 생성
#         SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
#         SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
#             #만든 모델에 train set 다시 넣어 trainset rmse 구하기 위해 만든것
#         SVR_Best_train_test = SVR_Best_model.predict(X_train)
#         SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
#             #훈련셋 RMSE & MAPE
#         SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
#         SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
#             #시험셋 RMSE & MAPE
#         SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
#         SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
#         SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
#         # 값들을 위에 지정한 list 에다 다 저장
#         temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
#         temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
#         temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
#         temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
#         temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
            

#     #     if SVR_Best_min>SVR_Best_MAPE_test:
#     #         SVR_Best_min = SVR_Best_MAPE_test
#     #         SVR_Best_TS = ts
#     #         SVR_Best_A = a
#     #         SVR_Best_B = b
#     #         SVR_Best_C = count+1

#         count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , test_size: {} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
#     SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
#     SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))


In [14]:
# SVR1=[]
# SVR2=[]
# SVR3=[]
# SVR4=[]
# SVR5=[]

# for i in range(50):
#     k=i//10
#     if k==0:
#         SVR1.append(SVR_Best_total_data[i])
#     if k==1:
#         SVR2.append(SVR_Best_total_data[i])
#     if k==2:
#         SVR3.append(SVR_Best_total_data[i])
#     if k==3:
#         SVR4.append(SVR_Best_total_data[i])
#     if k==4:
#         SVR5.append(SVR_Best_total_data[i])

    
# SVR_s1 = pd.Series(SVR1,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 1')

# SVR_s2 = pd.Series(SVR2,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 2')
# SVR_s3 = pd.Series(SVR3,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 3')
# SVR_s4 = pd.Series(SVR4,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 4')
# SVR_s5 = pd.Series(SVR5,
# index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
#        'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
#        'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
# name='Model 5')

# SVR_best_total_dataframe_linear = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
# print("SVR best 5 models")
# SVR_best_total_dataframe_linear

### 3. Linear Regression ... <br>
what parameters are there here?

In [14]:
times=10
count=0

# best_LR_RMSE_diff=100
# best_LR_RMSE_train=0
# best_LR_RMSE_test=0
# best_LR_MAPE_train=0
# best_LR_MAPE_test=0
LR_RMSE_train_list =[]
LR_RMSE_test_list =[]
LR_MAPE_train_list =[]
LR_MAPE_test_list =[]
LR_par_diff_list=[]
LR_par_tsize_list=[]
LR_min_vars_tsize_list=[] 

print("----progress...----")

for ts in test_size:
    for i in range(times):
        random.seed(a*10)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        LR_model=LinearRegression()
        LR_model.fit(X_train, Y_train)
        LR_prediction_test = LR_model.predict(X_test)
        LR_prediction_test = LR_prediction_test.astype('int')
        LR_train_test = LR_model.predict(X_train)
        LR_train_test = LR_train_test.astype('int')
    
        #훈련셋 RMSE & MAPE
        LR_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_train_test)).astype('int')
        LR_MAPE_train = round(np.mean(np.abs((Y_train - LR_train_test) / Y_train)) * 100 ,2)
            
        #시험셋 RMSE & MAPE
        LR_RMSE_test =np.sqrt(mean_squared_error(Y_test, LR_prediction_test)).astype('int')
        LR_MAPE_test = round(np.mean(np.abs((Y_test - LR_prediction_test) / Y_test)) * 100 ,2)
        
        # 값들을 위에 지정한 list 에다 다 저장
        LR_RMSE_train_list.append(LR_RMSE_train)
        LR_RMSE_test_list.append(LR_RMSE_test)
        LR_MAPE_train_list.append(LR_MAPE_train)
        LR_MAPE_test_list.append(LR_MAPE_test)
            
        LR_RMSE_diff = np.abs((LR_RMSE_test-LR_RMSE_train)/LR_RMSE_test *100)
            
        LR_par_diff_list.append(LR_RMSE_diff)
        LR_par_tsize_list.append(ts)
            
#             if best_LR_RMSE_diff>LR_RMSE_diff:
#                 best_LR_RMSE_diff = LR_RMSE_diff
#                 best_LR_RMSE_train = LR_RMSE_train
#                 best_LR_RMSE_test = LR_RMSE_test
#                 best_LR_MAPE_train = LR_MAPE_train
#                 best_LR_MAPE_test = LR_MAPE_test
#                 LR_C = count+1
#                 LR_TS = ts
#                 LR_A = a
#                 LR_B = b
         
                
        count+=1
        progress = round(count/(ts_len*times)*100,2)
            
        print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
        "---  train RMSE : {} , test RMSE : {}\n"
        "---  train MAPE : {} , test MAPE : {}"
                .format(progress,count,ts,LR_RMSE_diff,LR_RMSE_train,LR_RMSE_test,LR_MAPE_train,LR_MAPE_test))
            

# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} ,  \n"
#       "---  train RMSE : {} , test RMSE : {} , difference(%) : {:.2f} \n"
#       "---  train MAPE : {} , test MAPE : {}"
#       .format(LR_C,LR_TS,best_LR_RMSE_train,best_LR_RMSE_test,best_LR_RMSE_diff,best_LR_MAPE_train,best_LR_MAPE_test))
                


for i in range(5):
    val = min(LR_par_diff_list)
    index = LR_par_diff_list.index(val)
    LR_min_vars_tsize_list.append(LR_par_tsize_list.pop(index))


----progress...----
 2.00% , case:1 , test_size: 0.1 , difference(%) : 5.73
 ---  train RMSE : 1625 , test RMSE : 1537
---  train MAPE : 9.15 , test MAPE : 8.77
 4.00% , case:2 , test_size: 0.1 , difference(%) : 16.26
 ---  train RMSE : 1586 , test RMSE : 1894
---  train MAPE : 9.03 , test MAPE : 10.4
 6.00% , case:3 , test_size: 0.1 , difference(%) : 11.70
 ---  train RMSE : 1633 , test RMSE : 1462
---  train MAPE : 9.13 , test MAPE : 9.43
 8.00% , case:4 , test_size: 0.1 , difference(%) : 30.86
 ---  train RMSE : 1539 , test RMSE : 2226
---  train MAPE : 8.95 , test MAPE : 9.87
 10.00% , case:5 , test_size: 0.1 , difference(%) : 29.96
 ---  train RMSE : 1545 , test RMSE : 2206
---  train MAPE : 8.97 , test MAPE : 10.04
 12.00% , case:6 , test_size: 0.1 , difference(%) : 10.05
 ---  train RMSE : 1631 , test RMSE : 1482
---  train MAPE : 9.14 , test MAPE : 8.98
 14.00% , case:7 , test_size: 0.1 , difference(%) : 11.99
 ---  train RMSE : 1634 , test RMSE : 1459
---  train MAPE : 9.14 , 

In [15]:
print("PARAMETERS FOR BEST LR...")
for i in range(5):
    print("test_size : {} ".format(LR_min_vars_tsize_list[i]))

print("...testing...")

progress=0
count=0
LR_Best_RMSE_train_list=[]
LR_Best_RMSE_test_list=[]
LR_Best_MAPE_train_list=[]
LR_Best_MAPE_test_list=[]
LR_Best_RMSE_diff_list=[]

LR_Best_total_data=[]

for i in range(5):
    temp_LR_Best_RMSE_train_list=[]
    temp_LR_Best_RMSE_test_list=[]
    temp_LR_Best_MAPE_train_list=[]
    temp_LR_Best_MAPE_test_list=[]
    temp_LR_Best_RMSE_diff_list=[]
    
    for k in range(test_times):
        ts = LR_min_vars_tsize_list[i]
        random.seed((k+1)*10)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts)

        LR_Best_model=LinearRegression()
        LR_Best_model.fit(X_train, Y_train)
        LR_Best_prediction_test = LR_Best_model.predict(X_test)
        LR_Best_prediction_test = LR_Best_prediction_test.astype('int')
        LR_Best_train_test = LR_Best_model.predict(X_train)
        LR_Best_train_test = LR_Best_train_test.astype('int')

        #훈련셋 RMSE & MAPE
        LR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_Best_train_test)).astype('int')
        LR_Best_MAPE_train = round(np.mean(np.abs((Y_train - LR_Best_train_test) / Y_train)) * 100 ,2)

        #시험셋 RMSE & MAPE
        LR_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, LR_Best_prediction_test)).astype('int')
        LR_Best_MAPE_test = round(np.mean(np.abs((Y_test - LR_Best_prediction_test) / Y_test)) * 100 ,2)

        LR_Best_RMSE_diff = np.abs((LR_Best_RMSE_test-LR_Best_RMSE_train)/LR_Best_RMSE_test *100)
        
        # 값들을 위에 지정한 list 에다 다 저장
        temp_LR_Best_RMSE_train_list.append(LR_Best_RMSE_train)
        temp_LR_Best_RMSE_test_list.append(LR_Best_RMSE_test)
        temp_LR_Best_MAPE_train_list.append(LR_Best_MAPE_train)
        temp_LR_Best_MAPE_test_list.append(LR_Best_MAPE_test)
        temp_LR_Best_RMSE_diff_list.append(LR_Best_RMSE_diff)
            

    #     if LR_Best_min>LR_Best_MAPE_test:
    #         LR_Best_min = LR_Best_MAPE_test
    #         LR_Best_TS = ts
    #         LR_Best_A = a
    #         LR_Best_B = b
    #         LR_Best_C = count+1

        count+=1
        progress = round(count/(test_times*5)*100,2)
        print(" {:.2f}% , case:{} , test_size: {} , difference(%) : {:.2f}\n "
                "---  train RMSE : {} , test RMSE : {}\n"
                "---  train MAPE : {} , test MAPE : {}"
                      .format(progress,count,ts,LR_Best_RMSE_diff,LR_Best_RMSE_train,LR_Best_RMSE_test,LR_Best_MAPE_train,LR_Best_MAPE_test))
    
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(mean(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_train_list))
    LR_Best_total_data.append(std(temp_LR_Best_MAPE_test_list))
    LR_Best_total_data.append(mean(temp_LR_Best_RMSE_diff_list))
    LR_Best_total_data.append(std(temp_LR_Best_RMSE_diff_list))
    

#     LR_Best_RMSE_train_list.append(temp_LR_Best_RMSE_train_list)
#     LR_Best_RMSE_test_list.append(temp_LR_Best_RMSE_test_list)
#     LR_Best_MAPE_train_list.append(temp_LR_Best_MAPE_train_list)
#     LR_Best_MAPE_test_list.append(temp_LR_Best_MAPE_test_list)
#     LR_Best_RMSE_diff_list.append(temp_LR_Best_RMSE_diff_list)
    
    
# print("\n\n")
# print("Best Case")
# print("case:{} , test_size: {} , Tree_num: {} , Var_num: {} ,   ---  MAPE : {}".format(LR_Best_C,LR_Best_TS,LR_Best_A,LR_Best_B,LR_Best_min))



PARAMETERS FOR BEST LR...
test_size : 0.15 
test_size : 0.2 
test_size : 0.2 
test_size : 0.2 
test_size : 0.2 
...testing...
 2.00% , case:1 , test_size: 0.15 , difference(%) : 17.78
 ---  train RMSE : 1568 , test RMSE : 1907
---  train MAPE : 9.07 , test MAPE : 8.66
 4.00% , case:2 , test_size: 0.15 , difference(%) : 8.68
 ---  train RMSE : 1600 , test RMSE : 1752
---  train MAPE : 8.91 , test MAPE : 10.41
 6.00% , case:3 , test_size: 0.15 , difference(%) : 24.49
 ---  train RMSE : 1662 , test RMSE : 1335
---  train MAPE : 9.24 , test MAPE : 8.64
 8.00% , case:4 , test_size: 0.15 , difference(%) : 2.40
 ---  train RMSE : 1623 , test RMSE : 1585
---  train MAPE : 9.21 , test MAPE : 8.91
 10.00% , case:5 , test_size: 0.15 , difference(%) : 0.25
 ---  train RMSE : 1625 , test RMSE : 1629
---  train MAPE : 8.96 , test MAPE : 10.04
 12.00% , case:6 , test_size: 0.15 , difference(%) : 1.41
 ---  train RMSE : 1614 , test RMSE : 1637
---  train MAPE : 9.0 , test MAPE : 9.62
 14.00% , case:7 

In [16]:


LR1=[]
LR2=[]
LR3=[]
LR4=[]
LR5=[]

for i in range(50):
    k=i//10
    if k==0:
        LR1.append(LR_Best_total_data[i])
    if k==1:
        LR2.append(LR_Best_total_data[i])
    if k==2:
        LR3.append(LR_Best_total_data[i])
    if k==3:
        LR4.append(LR_Best_total_data[i])
    if k==4:
        LR5.append(LR_Best_total_data[i])
    
LR_s1 = pd.Series(LR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

LR_s2 = pd.Series(LR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
LR_s3 = pd.Series(LR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
LR_s4 = pd.Series(LR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
LR_s5 = pd.Series(LR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

LR_best_total_dataframe = pd.concat([LR_s1,LR_s2,LR_s3,LR_s4,LR_s5],axis=1)

LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1619.000000,1620.900000,1600.800000,1606.500000,1606.700000
RMSE test Mean,1615.800000,1665.400000,1685.300000,1731.300000,1663.300000
RMSE train Stdev,26.717036,31.586231,40.686116,48.380265,37.733407
RMSE test Stdev,162.157824,296.566418,166.479458,342.215736,147.539181
MAPE train Mean,9.075000,9.095000,9.123000,9.073000,9.077000
MAPE test Mean,9.319000,9.518000,8.981000,9.544000,9.223000
MAPE train Stdev,0.121099,0.182222,0.108540,0.254325,0.114983
MAPE test Stdev,0.656954,0.975108,0.431288,1.228847,0.309743
RMSE diff(%) Mean,9.123525,9.111046,10.935127,12.961350,8.855449
RMSE diff(%) Stdev,8.497705,11.171290,6.417689,11.405193,6.777533


In [15]:
RF_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,5901.000000,3080.200000,1934.200000,1375.800000,5836.400000
RMSE test Mean,5789.300000,3140.900000,2052.900000,1567.100000,5940.800000
RMSE train Stdev,90.743595,103.052220,36.790760,26.656331,64.417699
RMSE test Stdev,90.743595,103.052220,36.790760,26.656331,64.417699
MAPE train Mean,20.080000,10.990000,6.770000,4.572000,19.839000
MAPE test Mean,19.912000,11.094000,7.117000,5.005000,19.847000
MAPE train Stdev,0.452482,0.395929,0.144291,0.088747,0.176887
MAPE test Stdev,0.452482,0.395929,0.144291,0.088747,0.176887
RMSE diff(%) Mean,4.380071,4.623703,6.289273,11.936670,5.320058
RMSE diff(%) Stdev,2.896110,2.752326,3.630406,5.717231,3.325602


In [16]:
SVR_best_total_dataframe_rbf

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,2081.600000,1978.000000,2686.300000,1430.200000,2126.300000
RMSE test Mean,2189.000000,2188.000000,2720.700000,1760.600000,2174.800000
RMSE train Stdev,39.441602,25.262621,43.660165,42.300827,20.966879
RMSE test Stdev,39.441602,25.262621,43.660165,42.300827,20.966879
MAPE train Mean,4.660000,4.501000,5.588000,3.636000,4.687000
MAPE test Mean,5.207000,5.177000,6.106000,4.596000,5.221000
MAPE train Stdev,0.038987,0.061392,0.067941,0.050239,0.052163
MAPE test Stdev,0.038987,0.061392,0.067941,0.050239,0.052163
RMSE diff(%) Mean,9.410394,9.154732,7.936839,17.593459,7.913053
RMSE diff(%) Stdev,5.801017,6.132544,3.108907,10.621967,6.660207


In [17]:
LR_best_total_dataframe

,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,3422.000000,3429.600000,3414.200000,3427.200000,3430.800000
RMSE test Mean,3454.800000,3411.200000,3502.500000,3427.400000,3417.800000
RMSE train Stdev,27.254357,27.408758,19.301813,29.219172,25.443270
RMSE test Stdev,27.254357,27.408758,19.301813,29.219172,25.443270
MAPE train Mean,11.960000,12.000000,11.962000,11.986000,12.013000
MAPE test Mean,12.055000,11.768000,12.197000,12.058000,11.930000
MAPE train Stdev,0.092952,0.078230,0.088068,0.091345,0.104312
MAPE test Stdev,0.092952,0.078230,0.088068,0.091345,0.104312
RMSE diff(%) Mean,4.427254,4.776021,3.596998,3.598775,3.194342
RMSE diff(%) Stdev,2.983396,2.484808,2.436418,2.372767,1.820745
